In [43]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, Timer


from lion_pytorch import Lion

torch.manual_seed(42)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
    print("cuda", torch.cuda.is_available())
    print(torch.cuda.get_device_name(0))
    torch.cuda.empty_cache()
else:
    print("CUDA is not available.")

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pytorch_lightning.trainer.connectors.data_connector")
warnings.filterwarnings("ignore", category=UserWarning, module="lightning_fabric.plugins.environments.slurm")

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

torch.set_float32_matmul_precision('medium')

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from utils.add_skipatom import add_skipatom_features_to_dataset
from utils.utils import save_trial_to_csv, evaluate_model, create_hyperopt_dir, MoleculeDataModule
from utils.train import MoleculeModel

cuda True
NVIDIA GeForce RTX 3090


In [34]:
data = torch.load(f'../data/QM_137k.pt')
dataset = data[:1000]

In [35]:
dataset = add_skipatom_features_to_dataset(dataset, min_count=2e7, top_n=4, device='cpu', progress_bar=True, scaler=StandardScaler())
dataset[0]

Add Skipatom features: 100%|██████████| 1000/1000 [00:02<00:00, 359.58it/s]


Data(x=[31, 163], edge_index=[2, 64], edge_attr=[64, 14], y=[31], smiles='CNC(=S)N/N=C/c1c(O)ccc2ccccc12')

In [46]:
in_features = dataset[0].x.shape[1]
out_features = 1
edge_attr_dim = dataset[0].edge_attr.shape[1]

preprocess_hidden_features = [1700] * 9
postprocess_hidden_features = [128, 128]

cheb_hidden_features = [128, 128]
K = [10, 16]
cheb_normalization = ['sym', 'sym']

optimizer_class = Lion
learning_rate = 2.2e-5
weight_decay = 3e-5
step_size = 80
gamma = 0.2
max_epochs = 100
patience = 5
batch_size = 128

batch_size = 1024  
num_workers = 8  

data_module = MoleculeDataModule(dataset, batch_size=batch_size, num_workers=num_workers)

dropout_rates = [0.0] * (len(preprocess_hidden_features) + len(postprocess_hidden_features))
activation_fns = [nn.PReLU] * (len(preprocess_hidden_features) + len(postprocess_hidden_features))
use_batch_norm = [True] * (len(preprocess_hidden_features) + len(postprocess_hidden_features))

model = MoleculeModel(
    atom_in_features=in_features,
    edge_attr_dim=edge_attr_dim,
    preprocess_hidden_features=preprocess_hidden_features,
    cheb_hidden_features=cheb_hidden_features,
    K=K,
    cheb_normalizations=cheb_normalization,
    dropout_rates=dropout_rates,
    activation_fns=activation_fns,
    use_batch_norm=use_batch_norm,
    postprocess_hidden_features=postprocess_hidden_features,
    out_features=out_features,
    optimizer_class=optimizer_class,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    step_size=step_size,
    gamma=gamma,
    batch_size=batch_size,
    metric='rmse'
)



print("Model:\n", model)

checkpoint_callback = ModelCheckpoint(monitor='val_loss', mode='min', save_top_k=1, verbose=True)
early_stop_callback = EarlyStopping(monitor='val_loss', patience=patience, verbose=True, mode='min')
timer = Timer()
logger = pl.loggers.TensorBoardLogger('tb_logs', name='MolModel')

trainer = pl.Trainer(
    max_epochs=max_epochs,
    enable_checkpointing=False,
    callbacks=[early_stop_callback, timer],
    enable_progress_bar=False,
    logger=logger,
    accelerator='gpu',
    devices=1,
)
trainer.fit(model, data_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: tb_logs/MolModel
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type       | Params
-------------------------------------------------
0 | atom_preprocess   | ModuleList | 8.6 M 
1 | cheb_convolutions | ModuleList | 1.6 M 
2 | postprocess       | ModuleList | 33.5 K
3 | output_layer      | Linear     | 129   
-------------------------------------------------
10.2 M    Trainable params
0         Non-trainable params
10.2 M    Total params
40.809    Total estimated model params size (MB)


Model:
 MoleculeModel(
  (atom_preprocess): ModuleList(
    (0): AtomEdgeInteraction(
      (interaction): Linear(in_features=177, out_features=1024, bias=True)
    )
    (1): Sequential(
      (atom_linear_1): Linear(in_features=1024, out_features=1024, bias=True)
      (atom_bn_1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (atom_activation_1): PReLU(num_parameters=1)
      (atom_dropout_1): Dropout(p=0.0, inplace=False)
    )
    (2): Sequential(
      (atom_linear_2): Linear(in_features=1024, out_features=1024, bias=True)
      (atom_bn_2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (atom_activation_2): PReLU(num_parameters=1)
      (atom_dropout_2): Dropout(p=0.0, inplace=False)
    )
    (3): Sequential(
      (atom_linear_3): Linear(in_features=1024, out_features=1024, bias=True)
      (atom_bn_3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (atom

Metric val_loss improved. New best score: 0.217
Monitored metric val_loss did not improve in the last 5 records. Best score: 0.217. Signaling Trainer to stop.


In [37]:
seconds = timer.time_elapsed()
h, m, s = int(seconds // 3600), int((seconds % 3600) // 60), int(seconds % 60)

print(f"Время обучения: {h}:{m:02d}:{s:02d}")


Время обучения: 0:01:56
